<a target="_blank" href="https://colab.research.google.com/github/univiemops/tewa1-computational-cognition/blob/main/03%20Resampling.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Tutorial 3 - Resampling methods and Data Visualization

*Written and revised by Jozsef Arato, Mengfan Zhang, Dominik Pegler*  
Computational Cognition Course, University of Vienna  
https://github.com/univiemops/tewa1-computational-cognition

---

## This week's lab:

We will introduce you to two common resampling methods: **Bootstrapping** and **Permutation test** (an alternative to the t-test) with data visualization of them. Resampling techniques are powerful tools for estimating the sampling distribution of a statistic, making inferences about population parameters, validating models, and other situations where traditional parametric assumptions are not met or when exact analytical solutions are impractical. 

In this notebook, we have included many explanations as comments in the code cell. Please read them carefully instead of just pressing the run button.  

**Learning goals:** \
When finishing this tutorial, you should be able to ...
* use `random` functions to generate samples
* perform a bootstrap and estimate the variance 
* perform a permutation test and determine the statistical significance of models


**Estimated time to complete:** 2 hours \
**Deadline:** Next Wednesday, 10:00


# 1. Sampling

Sampling is the process of selecting a subset of values or items from a larger population. Python has various libraries to do sampling, a commonly used one is `numpy.random` module. Let's import the libraries first and have a look at some useful functions that we will use a lot in this course (or have used before).

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np

Below, we randomly sample items from an array. This means that each item in the array has an equal chance of being selected:

In [ ]:
my_array = np.arange(20)

# Without replacement
# Selected item will not be in the “pool” for selection
sample_1 = np.random.choice(my_array)
print("Randomly chosen item without replacement:", sample_1)

# With replacementare
# Selected item put back into the "pool" before another item is selected
sample_2 = np.random.choice(my_array, size=5, replace=True)
print("Randomly chosen items with replacement:", sample_2)

You can also generate random samples from a given distribution:

In [ ]:
# Randomly sample from a uniform distribution
sample_3 = np.random.uniform(low=0.0, high=10, size=10)
print("Randomly chosen items from a uniform distribution:", "\n", sample_3, "\n")

# Randomly sample from a normal distribution
sample_4 = np.random.normal(
    loc=0.0, scale=1.0, size=10
)  # loc and scale are the mean and sd of the distribution
print("Randomly chosen items from a noraml distribution:", "\n", sample_4, "\n")

# Randomly sample from a beta distribution
sample_5 = np.random.beta(a=0.5, b=0.5, size=10)
print("Randomly chosen items from a beta distribution:", "\n", sample_5)

It's also possible to generate a centrain format of numbers with a range:

In [ ]:
# Generate random integers within the range of low (inclusive) to high (exclusive)
sample_6 = np.random.randint(low=5, high=17, size=10)
print("Randomly chosen integers from 5 to 17:", "\n", sample_6, "\n")

# Generate floats within the range from 0 (inclusive) to 1 (exclusive)
sample_7 = np.random.random_sample(size=10)
print("Randomly chosen floats from 0 to 1:", "\n", sample_7)

Another useful function is `random.shuffle`, you can run the code below several times to figure out what this function does. 

In [ ]:
my_array = np.arange(20)
print("The original array:", my_array)

np.random.shuffle(my_array)
print("After randomly shuffling:", my_array)

The last thing we'd like to introduce here is `random.seed`, which we already used for last week. Random seed is used to generate pseudo-random numbers. If you provide the same seed value, you'll get the same sequence of random numbers each time you run your code. You can change the seed value back and forth, run the code below several times to see how it works. 

In [ ]:
np.random.seed(1)
sample_8 = np.random.random_sample(size=10)
print("After randomly shuffling with setting a seed:", sample_8)

# 2. Bootstrapping

Imagine you conducted an experiment in which you collected reaction time data from two small groups of participants under different manipulations. You compare the means of these two groups and find a difference, and you think the difference is due to the manipulation. However, it is also possible that one or two participants in one group didn't sleep well last night, weren't in good health, weren't paying attention, or some other random thing that you can't control. One thing you can do is repeat the experiment (maybe over and over again) to try to rule out the effect of random things. Fortunately, you have another much easier and less expensive option, which is to bootstrap. 

Bootstrapping is a technique used to estimate the sampling distribution of almost any statistic by repeatedly sampling with replacement from the observed data. It gives you a sense of what might happen if you repeated an experiment many times, and allows you to assess the variability and uncertainty associated with the statistic. Now let's perform bootstrapping procedures step by step:

In [ ]:
# Suppose we have some reaction time data from an experiment
np.random.seed(213)
rt_data = np.random.randint(150, 600, 20)
exp_mean = rt_data.mean()
print("Experimental reaction time data:", "\n", rt_data)
print("Mean of experimental reaction time", "\n", exp_mean)

In [ ]:
# Perform bootstrapping sampling
n_samples = 100  # perform the sampling 100 times, it works like repeating the experiment 100 times
bootstrap_means = np.zeros(n_samples)

for i in range(n_samples):
    # Step 1: Sampling with replacement to create a bootstrap sample
    current_bootstrap_sample = np.random.choice(
        rt_data, size=rt_data.size, replace=True
    )

    # Step 2: Calculate the statistic of interest for each sample
    current_mean = np.mean(current_bootstrap_sample)

    # Step 3: Store each sample statistic for estimating the sampling distribution
    bootstrap_means[i] = current_mean

# Step 4: Drawing Inferences
estimated_mean = np.mean(bootstrap_means)
estimated_std = np.std(bootstrap_means)

print("Estimated bootstrap mean :", estimated_mean)
print("Standard error of the estimate:", estimated_std)

In [ ]:
# Plot the sampling distribution of the statisitc
plt.hist(bootstrap_means)
plt.axvline(x=exp_mean, color="r", label="exp_mean")
plt.ylim((0, 30))
plt.xlabel("Means")
plt.ylabel("Frequency")
plt.title("Distributions", fontsize=15)
plt.legend(["exp_mean", "bootstrap_mean"])
plt.show()

Great! We just did our first bootstrap! In real practice, instead of writing a loop, Python `Scipy.stats` module provides the `bootstrap` function to do it easily. `Scipy` bulids on the functionality of `NumPy` but extends it by offering a wide range of specialized functions and algorithms for scientific and technical computing. 

In [ ]:
from scipy import stats

bootstrap_out = stats.bootstrap(
    data=(rt_data,), statistic=np.mean, n_resamples=100, axis=0
)  # check out ?bootstrap as well
print(bootstrap_out)

Can you do a bootstrap with 50 samples, calculate the standard deviation and store it in the variable "bootstrap_std"? It doesn't matter if you use the loop or the function.

In [ ]:
# YOUR CODE HERE

# 3. Permutation test

A permutation test is a **non-parametric** method used to assess the significance of an observed result by randomly shuffling the data and recalculating the statistic of interest multiple times. Compared to parametric methods (e.g., t-test, ANOVA), non-parametric methods do not have assumptions about the underlying distribution or the homogeneity of variance, but only assume the exchangeability within the data. Exchagenability means the prior values won't affect the future outcomes. For example, if you toss a coin, previous outcomes won't affect which side you will get this time. However, if you have time series data or spatial data with autocorrelation, a permutation test may not be appropriate. In general, permutation is robust and very useful when the assumptions of parametric test are not met, or when the sample size is small. \
Let's have a look at how to perform a permutation test using two reaction times datasets.

In [ ]:
# Below, we have reaction time data for the treatment and control groups, using a between subject design
treatment = np.array(
    [
        444.48703626,
        420.71413104,
        482.04432447,
        380.46896668,
        420.56864234,
        474.09130417,
        414.9748433,
        450.15423802,
        436.53977461,
        500.12705411,
        405.00705696,
        419.3141794,
        460.46096974,
        450.54358948,
        420.93431563,
        467.40481135,
        510.84094939,
        482.61924772,
        480.32638462,
        510.76756724,
    ]
)

control = np.array(
    [
        420.1243685,
        501.25211241,
        454.37132587,
        600.39850065,
        501.79657108,
        481.94197109,
        469.51703441,
        449.82747137,
        450.98838458,
        477.15878941,
        570.00039675,
        460.18766471,
        432.70480616,
        480.38394358,
        478.46070285,
        485.71067427,
        487.91937261,
        505.86604195,
        495.8480102,
        480.9547509,
    ]
)

Could you do some data exploration on these two groups in the cell below:

1. Calculate the mean and standard deviation for the treatment and control groups and store the results in the variables "tmt_mean", "cl_mean", "tmt_sd", "cl_sd", respectively.
2. Calculate the absolute difference in means between the groups and store it in the "true_diff" variable.
3. Create two subplots, each plot showing a histogram for each group. You can customize your plot, but be sure to set the same ranges for the x (370, 610) and y (0, 8) axes for both plots. We provide some code to do this, and compare your plot with the plot (we set 10 bins for histograms) from the test cell.

In [ ]:
# YOUR CODE HERE

# Subplots
plt.figure(figsize=(12, 5))  # create a new figure with a certain size
plt.subplot(
    1, 2, 1
)  # subplot(nrows, ncols, index), create subplots on a grid with nrows rows and ncols columns, the current plot take the index position
# YOUR CODE HERE: plot a histogram for the treatment group

plt.subplot(1, 2, 2)
# YOUR CODE HERE: plot a histogram for the control group

plt.show()

<div class="alert alert-info"> The cell below tests if your code is correct or not. Just run the cell, and it will give you an error message or a compliment.</div>

In [ ]:
# Test cell
tests = np.load("Answers/week3.npy", allow_pickle=True)
out = [tmt_mean, cl_mean, tmt_sd, cl_sd, true_diff]

try:
    assert (out == tests[0:5]).all()
except AssertionError as msg:
    print("At least one variable was not correctly calculated, check your code again!")
    raise (msg)
else:
    print("Good job!")

In [ ]:
# Run the cell, and compare your plot with the plot blow
image = plt.imread("Answers/tmt_cl_plot.png")
plt.figure(figsize=(15, 6.25))
plt.imshow(image)
plt.axis("off")
plt.show()

We can also visualize the means with error bars showing the standard deviation. To do this, we use the `plt.errorbar` function: 

In [ ]:
plt.figure()
plt.errorbar(
    x=0, y=np.mean(treatment), yerr=np.std(treatment), marker="o", markersize=10
)
plt.errorbar(x=1, y=np.mean(control), yerr=np.std(control), marker="o", markersize=10)
plt.ylabel("reaction time (ms)")
plt.xlim([-1, 2])
plt.xticks(np.arange(2), ["Treatment", "Control"])
plt.show()

It's also very convenient to create a more advanced boxplot using the `plt.boxplot` function:

In [ ]:
plt.boxplot([treatment, control], labels=["Treatment", "Control"])
plt.title("Boxplot of Treatment and Control Groups")
plt.xlabel("Group")
plt.ylabel("Reaction time (ms)")
plt.show()

All right, enough data exploration and plots for now. Let's get back to the permutation test. We will show you how to implement it in the cell below, and make sure you understand each step clearly. 

In [ ]:
# Step 1: Calculate the observed statistic of interest
obs_mean_diff = np.abs(
    treatment.mean() - control.mean()
)  # our example compare the means of two groups

# Step 2: Combine the data from two groups
combined_data = np.concatenate((treatment, control))  # check out ?np.concatenate

# We initiate a loop here because we need to repeat step 3-5 many many times
n_permutations = 1000
permuted_means_diff = np.zeros(n_permutations)

for i in range(n_permutations):
    # Step 3: Randomly shuffle the combined data to mix up groups
    np.random.shuffle(combined_data)

    # Step 4: From the randomly shuffled data, generate 2 groups with the same size as the original groups
    perm_group1 = combined_data[: len(treatment)]
    perm_group2 = combined_data[len(treatment) :]

    # Step 5: Calculate the statistic of interest from the 2 permuted groups and store the result from each permutation
    permuted_means_diff[i] = perm_group1.mean() - perm_group2.mean()

# Step 6: Calculate the p-value as the proportion of:
# one-sided: permuted statistic greater than or less than your observed statistic
# two-sided: permuted statistic greater than your observed statistic and less than your negative observed statistic
p_value = (
    np.sum(
        np.sum(permuted_means_diff >= obs_mean_diff)
        + np.sum(permuted_means_diff <= -obs_mean_diff)
    )
    / n_permutations
)
print(p_value)

Now that we have the p-value from the permutation test, could you manually perform a traditional t-test and compare the result to the above permutation test?   
1. Calculate the t-statistic using the formula: $$ t = \frac{\bar{x_1} - \bar{x_2}}{\sqrt{\frac{s_1^2}{n_1 - 1} + \frac{s_2^2}{n_2 - 1}}} $$
2. Convert the t-statistic to the p-value using the function: `scipy.stats.t.sf(t, df)`. This function will give you a one-sided p-value, just multiply by 2 to get the two-sided p-value. 

In [ ]:
# YOUR CODE HERE

In the cell below, we perform a two-sample independent t-test, you can compare your calculation with it, they should be the same. 

In [ ]:
stats.ttest_ind(treatment, control, alternative="two-sided")

Once this is done, we can also evaluate the result of the permutations visually:

In [ ]:
plt.hist(permuted_means_diff, bins=100)  # distribution of permutated differences
plt.axvline(
    obs_mean_diff, color="red"
)  # mark the observed difference on the histogram with a vertical line
plt.axvline(-obs_mean_diff, color="red")
plt.axvline(
    np.percentile(permuted_means_diff, 5), color="black", linestyle="dashed"
)  # np.percentile calcultes the q-th% of the input array
plt.axvline(np.percentile(permuted_means_diff, 95), color="black", linestyle="dashed");

Okay, the last thing we want to tell you about the permutation test is that instead of writing a loop, `scipy.stats` also provides a function to do the permutation test easily:

In [ ]:
# We first write a function to determine the statistic to test
def statistic(x, y):
    return np.mean(x) - np.mean(y)


# Then call the permutation test function
per_test = scipy.stats.permutation_test(
    (treatment, control),
    statistic,
    permutation_type="independent",
    n_resamples=1000,
    alternative="two-sided",
)

print("Observed mean difference:", per_test.statistic)
print("p-value from the permutation test:", per_test.pvalue)

Can you do a permutation test on the correlation between the treatment and control groups with 1000 resamples"? It doesn't matter if you use the loop or the function. Hint: You may need the `np.corrcoef` function, check out how to use it. 

In [ ]:
# YOUR CORE HERE

## Homework 1

### Functions
now we are ready with the permutation test, now "re-cycle" your code from above, to make it into a function below.

This function should take 2 inputs (the 2 data-sets),
do the permuations as above, and return the permuted p-value.

Remember you can choose whatever variable names within the function, but best practice is not to use the same ones as for the code outside the function!

the function shoud not rely on any of the variables that are defined outside of the function: you can verify this by copy-pasting it into a new notebook and test it with some new data.




In [ ]:
def my_perm_test(#your code):
  #your code 
  #your code 
  return #your code 

 verify that your function works, with comparing it with your previous code
 

In [ ]:
# YOUR CODE

## now lets try the permutation test on some new data

In [ ]:
x1 = np.array(
    [
        200.48703626,
        420.71413104,
        482.04432447,
        380.46896668,
        420.56864234,
        474.09130417,
        414.9748433,
        450.15423802,
        436.53977461,
        500.12705411,
        405.00705696,
        419.3141794,
        460.46096974,
        450.54358948,
        420.93431563,
        467.40481135,
        510.84094939,
        482.61924772,
        480.32638462,
        860.56161,
    ]
)

x2 = np.array(
    [
        420.1243685,
        501.25211241,
        454.37132587,
        900.39850065,
        501.79657108,
        481.94197109,
        469.51703441,
        449.82747137,
        450.98838458,
        477.15878941,
        570.00039675,
        460.18766471,
        432.70480616,
        480.38394358,
        478.46070285,
        485.71067427,
        487.91937261,
        505.86604195,
        495.8480102,
        1500.5446,
    ]
)

explore these data-sets with histograms

now, we can easily use the function to compare other data-sets

for example X1 and X2 above
compare the result with the t-test

In [ ]:
my_perm_test(x1, x2)

log transform data before running the statistics

a common way to deal with data with large variability (eg: free viewing looking times) is to do a log transform.

1. make a scatter plot with the original data on the X and the raw data on the Y axis 
add both datasets to the same figure (using different colors)


2. run the permutation test and the t-test for the log-transformed datasets





### optional function parameters

until now we fixed the number of permutations in MyPermTest

make a new function, MyPermTestv2, with the only difference that there is a 3rd input variable, that controls the number of permutations


In [ ]:
def my_perm_test_v2(#your code):
    #your code
#your code

once this is done, make a visualization of how the permuted p value changes as you change the number of permutations


1. use `my_perm_test_v2`, to compare data1 and data2, with the number of permutations changing from 50 to 3000 in steps of 200  (use np.arange and a for cycle), at each step you only need to calcualte the permuted p-value
2. make a scatter plot, to visualize what you have calculate in 1.: the number of permutations on the x axis and the permuted p-value on the y-axis
3. add a horizontal line, that crosses the whole figure and shows the p-value obtained by  t-test

In [ ]:
# your code
# your code
# your code

### Bootstrapped confidence interval 

calculate the bootstrapped confidence interval for the mean of Data 1 and Data 2 with 2000 bootstraps. 
You can use np.random.choice ot perform sampling with replacemen- check the slided for more info.

after you have the 2000 sample means, you can use np.percentile to find the 95,99% confidence intervals

visualize what you found: if you use errobar, not that you can define positive and negative error differently (as a boostrapped confidence interval is not symmetric)

## Homework 2 (advanced)


### Data simulation and analysis

test permutation test and compare with the t-test with largely unequal sample sizes 

imagine a scenario where you research reaction times in a rare patient group
,so that you only have 5 patients.. obviously it is easier to get a large number of control participants, lets say 50

set mean=600 for patiens with SD of 100 (normal distribution)
set mean=550 for control with SD of 100 (uniform distribution)

1. make 2 data sets, patient group with N=5, the control group with N=50, 
2. compare them with the t-test
3. compare them using the permutation test
4. repeat this process multiple times, and systematically compare (with for cycle and visualization) the similarity between permuation and t-test result


